In [2]:
import sys
sys.path.append("../Common/")

import CommonYFinance, CommonBinance, CommonMT5, CommonMongoDBConnector, CommonBacktest

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Bước 1: Nhập Thư Viện và Chuẩn Bị Dữ Liệu


In [4]:
symbol = 'VCB.VN'
from_date = '2023-07-20'
to_date = '2023-12-14'
interval = '1d'
data = CommonYFinance.CommonYFinance.loaddataYFinance(symbol, from_date, to_date, interval)

[*********************100%%**********************]  1 of 1 completed


In [5]:
data

,Datetime,Open,High,Low,Close,Volume
0,2023-07-20,89331.078125,90093.140625,87722.265625,88907.703125,1038813
1,2023-07-21,88907.703125,88907.703125,88060.968750,88907.703125,2153467
2,2023-07-24,88992.382812,89246.398438,88484.335938,88569.007812,1079105
3,2023-07-26,91900.000000,93400.000000,91900.000000,93400.000000,1737418
4,2023-07-28,92700.000000,93000.000000,91800.000000,93000.000000,1207918
...,...,...,...,...,...,...
93,2023-12-07,85700.000000,86000.000000,85000.000000,85200.000000,2403460
94,2023-12-08,85200.000000,85500.000000,85000.000000,85000.000000,1293000
95,2023-12-11,85000.000000,85300.000000,84100.000000,84200.000000,2768100
96,2023-12-12,84700.000000,84700.000000,84000.000000,84200.000000,1148400


# Bước 2: Tải và Tiền Xử Lý Dữ Liệu


In [7]:
# X là các đặc trưng, y là biến mục tiêu
X = data.drop('Close', axis=1)
y = data['Close']

# Chia dữ liệu thành tập huấn luyện và kiểm thử
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuyển cột Datetime sang Timestamp
X_train['Datetime'] = X_train['Datetime'].astype(np.int64) // 10**9  # Chuyển thành timestamp
X_test['Datetime'] = X_test['Datetime'].astype(np.int64) // 10**9


# Bước 3: Huấn Luyện Mô Hình

In [8]:
# Huấn luyện mô hình Hồi Quy Tuyến Tính
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

# Huấn luyện mô hình Cây Quyết Định
model_dt = DecisionTreeRegressor(random_state=42)
model_dt.fit(X_train, y_train)


DecisionTreeRegressor(random_state=42)

# Bước 4: Dự Đoán và Đánh Giá Mô Hình


In [9]:
# Dự đoán trên tập kiểm thử
predictions_lr = model_lr.predict(X_test)
predictions_dt = model_dt.predict(X_test)

# Tính toán MSE và R-squared cho Hồi Quy Tuyến Tính
mse_lr = mean_squared_error(y_test, predictions_lr)
r2_lr = r2_score(y_test, predictions_lr)

# Tính toán MSE và R-squared cho Cây Quyết Định
mse_dt = mean_squared_error(y_test, predictions_dt)
r2_dt = r2_score(y_test, predictions_dt)

# In kết quả
print(f"Linear Regression - MSE: {mse_lr}, R2: {r2_lr}")
print(f"Decision Tree - MSE: {mse_dt}, R2: {r2_dt}")


Linear Regression - MSE: 169476.76142661128, R2: 0.9576251495200124
Decision Tree - MSE: 911540.7794067382, R2: 0.7720843618403794


# Bước 5: So Sánh và Phân Tích

Sử dụng các chỉ số MSE (Mean Squared Error) và R² (R-squared) để so sánh hiệu suất của hai mô hình.

# MSE thấp và R² cao cho thấy mô hình có hiệu suất tốt hơn.

Kết quả của việc đánh giá hai mô hình - Hồi Quy Tuyến Tính (Linear Regression) và Cây Quyết Định (Decision Tree) - trên tập kiểm thử của bạn cho thấy:

Hồi Quy Tuyến Tính:
MSE (Mean Squared Error): 169492.43. Đây là giá trị trung bình của bình phương sai số giữa giá trị dự đoán và giá trị thực tế. MSE càng thấp, mô hình càng chính xác.
R² (R-squared): 0.9576. Đây là tỉ lệ phần trăm biến đổi của biến mục tiêu được giải thích bởi mô hình. R² càng gần 1, mô hình càng phù hợp với dữ liệu.

Cây Quyết Định:
MSE: 911540.78. MSE cao hơn nhiều so với Hồi Quy Tuyến Tính, điều này cho thấy mô hình Cây Quyết Định có sai số dự đoán cao hơn.
R²: 0.7721. Dù vẫn khá cao, nhưng R² thấp hơn so với Hồi Quy Tuyến Tính, cho thấy mô hình này giải thích ít biến đổi của dữ liệu hơn.

Phân Tích và Kết Luận:
Dựa trên MSE và R², mô hình Hồi Quy Tuyến Tính (Linear Regression) có vẻ phù hợp hơn với dữ liệu kiểm thử so với mô hình Cây Quyết Định (Decision Tree). Nó có sai số thấp hơn và giải thích được một tỉ lệ lớn hơn biến đổi của dữ liệu.
Tuy nhiên, cần lưu ý rằng việc chọn mô hình phù hợp không chỉ dựa trên số liệu đánh giá mà còn phụ thuộc vào bối cảnh cụ thể của bài toán, bản chất của dữ liệu và yêu cầu về độ chính xác cũng như tính hiệu quả trong thực tiễn.
Nếu mô hình Cây Quyết Định có R² thấp và MSE cao, có thể xem xét việc điều chỉnh các tham số của mô hình như độ sâu của cây, số lượng lá tối thiểu, v.v., hoặc xem xét việc sử dụng một thuật toán khác.
Nhìn chung, kết quả cho thấy Hồi Quy Tuyến Tính là lựa chọn tốt hơn trong trường hợp này. Tuy nhiên, cần phải xem xét kỹ lưỡng hơn về dữ liệu và yêu cầu của bài toán để đưa ra quyết định cuối cùng.